# General Stuff
---

In [1]:
import os
import geopandas as gpd
import pandas as pd

import numpy as np

In [2]:
# Get script and dataset file paths.
SCRIPT_PATH = "/Users/Syrin/Documents/GitHub/AI4PublicHealth/"

# Read the road line data .shp file via geopandas and store as a pandas dataframe.
ROAD_LINE_PATH = os.path.join(SCRIPT_PATH, "Datasets/MZUZU_roads_lines_CORRECT.shp")
ROAD_LINE_DATA = gpd.read_file(ROAD_LINE_PATH)
ROAD_LINE_DATA = pd.DataFrame(ROAD_LINE_DATA)

INCLUDE_MULTILINE = False

# Create DataFrame
---

In [3]:
def road_line_processing(road_line_df):
    """Clean the .shp file that contains the route data. Create a second pandas data frame to store a processed
        version of the original data from the .shp file. """

    processed_data_line = []

    for rows in range(len(road_line_df.index)):
        coordinates_line = road_line_df.iloc[rows, 11]
        string_type = (type(coordinates_line))

        if INCLUDE_MULTILINE:
            if str(string_type) == "<class 'shapely.geometry.linestring.LineString'>":
                coordinates_line = list(coordinates_line.coords)

                start_longitude_line = coordinates_line[0][0]
                start_latitude_line = coordinates_line[0][1]
                end_longitude_line = coordinates_line[-1][0]
                end_latitude_line = coordinates_line[-1][1]

                processed_data_line.append(
                    (start_longitude_line, start_latitude_line, end_longitude_line, end_latitude_line))

            elif str(string_type) != "<class 'shapely.geometry.linestring.MultiLineString'>":
                for item in coordinates_line:
                    coordinates_line = item

                    coordinates_line = list(coordinates_line.coords)

                    start_longitude_line = coordinates_line[0][0]
                    start_latitude_line = coordinates_line[0][1]
                    end_longitude_line = coordinates_line[-1][0]
                    end_latitude_line = coordinates_line[-1][1]

                    processed_data_line.append(
                        (start_longitude_line, start_latitude_line, end_longitude_line, end_latitude_line))

            else:
                print("There is a unique string type that is neither LineString or MultiString:")
                print("    ", string_type)

        else:
            if str(string_type) == "<class 'shapely.geometry.linestring.LineString'>":
                coordinates_line = list(coordinates_line.coords)

                start_longitude_line = round(coordinates_line[0][0], 3)
                start_latitude_line = round(coordinates_line[0][1], 3)
                end_longitude_line = round(coordinates_line[-1][0], 3)
                end_latitude_line = round(coordinates_line[-1][1], 3)

                processed_data_line.append(
                    (start_latitude_line, start_longitude_line, end_latitude_line, end_longitude_line, np.array([])))

            else:
                continue

    processed_data_line = pd.DataFrame(processed_data_line)
    processed_data_line = processed_data_line.rename(
        columns={0: "Start Latitude", 1: "Start Longitude", 2: "End Latitude", 3: "End Longitude", 4: "Connection(s)"})

    return processed_data_line

road_line_nodes = road_line_processing(ROAD_LINE_DATA)
road_line_nodes

,Start Latitude,Start Longitude,End Latitude,End Longitude,Connection(s)
0,-11.391,33.985,-11.369,33.984,[]
1,-11.397,33.957,-11.394,34.048,[]
2,-11.391,33.960,-11.400,33.963,[]
3,-11.393,33.989,-11.410,33.993,[]
4,-11.388,34.046,-11.388,34.045,[]
...,...,...,...,...,...
5458,-11.449,33.971,-11.449,33.970,[]
5459,-11.450,33.970,-11.450,33.970,[]
5460,-11.437,33.964,-11.436,33.964,[]
5461,-11.446,33.967,-11.445,33.965,[]


# Testing
---

In [4]:
road_line_nodes.iloc[442]

Start Latitude    -11.409
Start Longitude    34.003
End Latitude      -11.399
End Longitude      33.995
Connection(s)          []
Name: 442, dtype: object

# Create Dicts
---

In [5]:
start_road_line_dict = {}
end_road_line_dict = {}

for index, row, in road_line_nodes.iterrows():
    start_lat_long_pair = (row["Start Latitude"], row["Start Longitude"])
    end_lat_long_pair = (row["End Latitude"], row["End Longitude"])
    
    if start_lat_long_pair not in start_road_line_dict:
        start_road_line_dict[start_lat_long_pair] = [index]
    else: 
        start_road_line_dict[start_lat_long_pair].append(index)
        
    if end_lat_long_pair not in end_road_line_dict:
        end_road_line_dict[end_lat_long_pair] = [index]
    else: 
        end_road_line_dict[end_lat_long_pair].append(index)

In [6]:
start_road_line_dict

{(-11.391, 33.985): [0],
 (-11.397, 33.957): [1],
 (-11.391, 33.96): [2],
 (-11.393, 33.989): [3, 98],
 (-11.388, 34.046): [4],
 (-11.408, 33.983): [5],
 (-11.402, 33.983): [6],
 (-11.399, 33.995): [7, 849],
 (-11.384, 33.966): [8],
 (-11.39, 33.962): [9],
 (-11.394, 34.048): [10],
 (-11.394, 33.998): [11, 64],
 (-11.38, 33.989): [12],
 (-11.382, 33.989): [13, 14, 15, 108],
 (-11.382, 33.984): [16],
 (-11.382, 33.985): [17, 18, 121],
 (-11.38, 33.985): [19, 22],
 (-11.376, 33.987): [20],
 (-11.379, 33.985): [21],
 (-11.374, 33.989): [23, 25],
 (-11.375, 33.987): [24],
 (-11.373, 33.987): [26, 30],
 (-11.375, 33.988): [27],
 (-11.371, 33.985): [28],
 (-11.37, 33.989): [29],
 (-11.379, 33.973): [31],
 (-11.398, 34.02): [32],
 (-11.391, 33.988): [33, 34, 89, 91, 92],
 (-11.391, 33.986): [35],
 (-11.39, 33.984): [36, 126, 127],
 (-11.39, 33.981): [37],
 (-11.39, 33.983): [38],
 (-11.388, 33.985): [39],
 (-11.387, 33.981): [40],
 (-11.389, 33.98): [41],
 (-11.389, 33.97): [42],
 (-11.393, 3

In [7]:
end_road_line_dict

{(-11.369, 33.984): [0],
 (-11.394, 34.048): [1, 60],
 (-11.4, 33.963): [2, 773],
 (-11.41, 33.993): [3, 991],
 (-11.388, 34.045): [4],
 (-11.393, 33.977): [5],
 (-11.391, 33.987): [6, 35],
 (-11.391, 33.988): [7, 92],
 (-11.392, 33.972): [8],
 (-11.392, 33.962): [9],
 (-11.381, 34.043): [10, 265],
 (-11.428, 34.003): [11, 297, 807],
 (-11.38, 33.988): [12, 19],
 (-11.38, 33.989): [13, 105],
 (-11.384, 33.988): [14, 39],
 (-11.382, 33.988): [15],
 (-11.382, 33.983): [16, 121],
 (-11.382, 33.984): [17],
 (-11.382, 33.987): [18, 109],
 (-11.376, 33.985): [20],
 (-11.379, 33.985): [21],
 (-11.377, 33.986): [22],
 (-11.376, 33.992): [23],
 (-11.374, 33.99): [24, 29],
 (-11.374, 33.988): [25],
 (-11.376, 33.981): [26],
 (-11.373, 33.987): [27],
 (-11.372, 33.983): [28],
 (-11.37, 33.989): [30, 237, 239],
 (-11.383, 33.975): [31],
 (-11.392, 34.025): [32],
 (-11.389, 33.991): [33],
 (-11.388, 33.986): [34],
 (-11.392, 33.987): [36],
 (-11.391, 33.982): [37],
 (-11.39, 33.985): [38],
 (-11.38

In [8]:
for index, row in road_line_nodes.iterrows():
    start_lat_long_pair = (row["Start Latitude"], row["Start Longitude"])
    end_lat_long_pair = (row["End Latitude"], row["End Longitude"])
    
    current_indexs = road_line_nodes.iat[index, 4]
    start_index_number = []
    end_index_number = []
    
    if start_lat_long_pair in start_road_line_dict:
        start_index_number = start_road_line_dict[start_lat_long_pair]


    if end_lat_long_pair in end_road_line_dict:
        end_index_number = end_road_line_dict[end_lat_long_pair]
        
    for indexs in np.append(start_index_number, end_index_number):
        if indexs != index and index not in current_indexs:
            print(index, start_index_number, end_index_number, current_indexs)
            current_indexs = np.append(current_indexs, indexs)
            road_line_nodes.iat[index, 4] = current_indexs

1 [1] [1, 60] []
2 [2] [2, 773] []
3 [3, 98] [3, 991] []
3 [3, 98] [3, 991] [98.]
6 [6] [6, 35] []
7 [7, 849] [7, 92] []
7 [7, 849] [7, 92] [849.]
10 [10] [10, 265] []
11 [11, 64] [11, 297, 807] []
11 [11, 64] [11, 297, 807] [64.]
11 [11, 64] [11, 297, 807] [ 64. 297.]
12 [12] [12, 19] []
13 [13, 14, 15, 108] [13, 105] []
13 [13, 14, 15, 108] [13, 105] [14.]
13 [13, 14, 15, 108] [13, 105] [14. 15.]
13 [13, 14, 15, 108] [13, 105] [ 14.  15. 108.]
14 [13, 14, 15, 108] [14, 39] []
14 [13, 14, 15, 108] [14, 39] [13.]
14 [13, 14, 15, 108] [14, 39] [13. 15.]
14 [13, 14, 15, 108] [14, 39] [ 13.  15. 108.]
15 [13, 14, 15, 108] [15] []
15 [13, 14, 15, 108] [15] [13.]
15 [13, 14, 15, 108] [15] [13. 14.]
16 [16] [16, 121] []
17 [17, 18, 121] [17] []
17 [17, 18, 121] [17] [18.]
18 [17, 18, 121] [18, 109] []
18 [17, 18, 121] [18, 109] [17.]
18 [17, 18, 121] [18, 109] [ 17. 121.]
19 [19, 22] [12, 19] []
19 [19, 22] [12, 19] [22.]
22 [19, 22] [22] []
23 [23, 25] [23] []
24 [24] [24, 29] []
25 [23, 25

291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407. 1408.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407. 1408. 1417.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407. 1408. 1417. 1419.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407. 1408. 1417. 1419. 1421.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 1448] [291] [1155. 1201. 1317. 1406. 1407. 1408. 1417. 1419. 1421. 1428.]
291 [291, 1155, 1201, 1317, 1406, 1407, 1408, 1417, 1419, 1421, 1428, 1444, 

408 [376, 408, 743, 744] [408, 410, 737] [376.]
408 [376, 408, 743, 744] [408, 410, 737] [376. 743.]
408 [376, 408, 743, 744] [408, 410, 737] [376. 743. 744.]
408 [376, 408, 743, 744] [408, 410, 737] [376. 743. 744. 410.]
409 [409, 438, 831] [409] []
409 [409, 438, 831] [409] [438.]
410 [410] [408, 410, 737] []
410 [410] [408, 410, 737] [408.]
411 [411, 413, 885, 886] [411] []
411 [411, 413, 885, 886] [411] [413.]
411 [411, 413, 885, 886] [411] [413. 885.]
413 [411, 413, 885, 886] [413] []
413 [411, 413, 885, 886] [413] [411.]
413 [411, 413, 885, 886] [413] [411. 885.]
415 [415] [415, 597] []
416 [416] [273, 332, 334, 360, 416, 435, 436] []
416 [416] [273, 332, 334, 360, 416, 435, 436] [273.]
416 [416] [273, 332, 334, 360, 416, 435, 436] [273. 332.]
416 [416] [273, 332, 334, 360, 416, 435, 436] [273. 332. 334.]
416 [416] [273, 332, 334, 360, 416, 435, 436] [273. 332. 334. 360.]
416 [416] [273, 332, 334, 360, 416, 435, 436] [273. 332. 334. 360. 435.]
420 [406, 420] [420] []
421 [421, 62

472 [472, 521, 532, 535] [462, 472, 483] [521.]
472 [472, 521, 532, 535] [462, 472, 483] [521. 532.]
472 [472, 521, 532, 535] [462, 472, 483] [521. 532. 535.]
472 [472, 521, 532, 535] [462, 472, 483] [521. 532. 535. 462.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] []
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519. 478.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519. 478. 482.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519. 478. 482. 484.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519. 478. 482. 484. 486.]
473 [473, 519] [473, 478, 482, 484, 486, 702, 703, 704, 705, 707, 712, 981, 982, 983, 984] [519. 478. 482. 484. 486. 702.]
473 [473, 519] [473, 

 981. 982. 983. 467. 695. 696. 697. 698.]
479 [476, 478, 479, 482, 483, 485, 486, 695, 702, 703, 704, 705, 708, 711, 712, 981, 982, 983] [467, 479, 695, 696, 697, 698, 699, 700, 701, 980] [476. 478. 482. 483. 485. 486. 695. 702. 703. 704. 705. 708. 711. 712.
 981. 982. 983. 467. 695. 696. 697. 698. 699.]
479 [476, 478, 479, 482, 483, 485, 486, 695, 702, 703, 704, 705, 708, 711, 712, 981, 982, 983] [467, 479, 695, 696, 697, 698, 699, 700, 701, 980] [476. 478. 482. 483. 485. 486. 695. 702. 703. 704. 705. 708. 711. 712.
 981. 982. 983. 467. 695. 696. 697. 698. 699. 700.]
479 [476, 478, 479, 482, 483, 485, 486, 695, 702, 703, 704, 705, 708, 711, 712, 981, 982, 983] [467, 479, 695, 696, 697, 698, 699, 700, 701, 980] [476. 478. 482. 483. 485. 486. 695. 702. 703. 704. 705. 708. 711. 712.
 981. 982. 983. 467. 695. 696. 697. 698. 699. 700. 701.]
480 [468, 480, 484, 689, 690, 692, 694, 696, 697, 698, 699, 700, 701, 706, 707, 980] [480, 489, 713, 2139, 2140, 2141, 2147, 2881, 2882, 2888, 2889] []

...

5339 [4873, 4875, 4876, 4877, 4919, 5328, 5329, 5339, 5342, 5344] [4835, 4872, 4873, 4874, 4875, 4876, 4877, 4880, 4918, 5329, 5337, 5339, 5342, 5344] [4873. 4875. 4876. 4877. 4919. 5328. 5329. 5342.]
5339 [4873, 4875, 4876, 4877, 4919, 5328, 5329, 5339, 5342, 5344] [4835, 4872, 4873, 4874, 4875, 4876, 4877, 4880, 4918, 5329, 5337, 5339, 5342, 5344] [4873. 4875. 4876. 4877. 4919. 5328. 5329. 5342. 5344.]
5339 [4873, 4875, 4876, 4877, 4919, 5328, 5329, 5339, 5342, 5344] [4835, 4872, 4873, 4874, 4875, 4876, 4877, 4880, 4918, 5329, 5337, 5339, 5342, 5344] [4873. 4875. 4876. 4877. 4919. 5328. 5329. 5342. 5344. 4835.]
5339 [4873, 4875, 4876, 4877, 4919, 5328, 5329, 5339, 5342, 5344] [4835, 4872, 4873, 4874, 4875, 4876, 4877, 4880, 4918, 5329, 5337, 5339, 5342, 5344] [4873. 4875. 4876. 4877. 4919. 5328. 5329. 5342. 5344. 4835. 4872.]
5339 [4873, 4875, 4876, 4877, 4919, 5328, 5329, 5339, 5342, 5344] [4835, 4872, 4873, 4874, 4875, 4876, 4877, 4880, 4918, 5329, 5337, 5339, 5342, 5344] [4873. 48

5376 [4394, 5049, 5050, 5054, 5376] [2008, 5030, 5037, 5122, 5132, 5353, 5376] [4394. 5049. 5050. 5054. 2008. 5030. 5037. 5122.]
5376 [4394, 5049, 5050, 5054, 5376] [2008, 5030, 5037, 5122, 5132, 5353, 5376] [4394. 5049. 5050. 5054. 2008. 5030. 5037. 5122. 5132.]
5377 [5055, 5056, 5377] [5057, 5058, 5377, 5382] []
5377 [5055, 5056, 5377] [5057, 5058, 5377, 5382] [5055.]
5377 [5055, 5056, 5377] [5057, 5058, 5377, 5382] [5055. 5056.]
5377 [5055, 5056, 5377] [5057, 5058, 5377, 5382] [5055. 5056. 5057.]
5377 [5055, 5056, 5377] [5057, 5058, 5377, 5382] [5055. 5056. 5057. 5058.]
5378 [4251, 5378] [5378] []
5379 [4238, 5105, 5379] [4237, 4250, 5105, 5106, 5379] []
5379 [4238, 5105, 5379] [4237, 4250, 5105, 5106, 5379] [4238.]
5379 [4238, 5105, 5379] [4237, 4250, 5105, 5106, 5379] [4238. 5105.]
5379 [4238, 5105, 5379] [4237, 4250, 5105, 5106, 5379] [4238. 5105. 4237.]
5379 [4238, 5105, 5379] [4237, 4250, 5105, 5106, 5379] [4238. 5105. 4237. 4250.]
5379 [4238, 5105, 5379] [4237, 4250, 5105, 510

5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402. 5404.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402. 5404. 4332.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402. 5404. 4332. 4531.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402. 5404. 4332. 4531. 4532.]
5403 [4535, 5287, 5392, 5402, 5403, 5404] [4332, 4531, 4532, 5392, 5402, 5403] [4535. 5287. 5392. 5402. 5404. 4332. 4531. 4532. 5392.]
5404 [4535, 5287, 5392, 5402, 5403, 5404] [5404] []
5404 [4535, 5287, 5392, 5402, 5403, 5404] [5404] [4535.]
5404 [4535, 5287, 5392, 5402, 5403, 5404] [5404] [4535. 5287.]
5404 [453

5424 [4848, 4996, 5363, 5424] [4846, 4849, 4850, 4851, 4856, 4858, 5133, 5424] [4848. 4996. 5363. 4846. 4849.]
5424 [4848, 4996, 5363, 5424] [4846, 4849, 4850, 4851, 4856, 4858, 5133, 5424] [4848. 4996. 5363. 4846. 4849. 4850.]
5424 [4848, 4996, 5363, 5424] [4846, 4849, 4850, 4851, 4856, 4858, 5133, 5424] [4848. 4996. 5363. 4846. 4849. 4850. 4851.]
5424 [4848, 4996, 5363, 5424] [4846, 4849, 4850, 4851, 4856, 4858, 5133, 5424] [4848. 4996. 5363. 4846. 4849. 4850. 4851. 4856.]
5424 [4848, 4996, 5363, 5424] [4846, 4849, 4850, 4851, 4856, 4858, 5133, 5424] [4848. 4996. 5363. 4846. 4849. 4850. 4851. 4856. 4858.]
5425 [4887, 4888, 4898, 4899, 4911, 5355, 5422, 5425] [5054, 5423, 5425] []
5425 [4887, 4888, 4898, 4899, 4911, 5355, 5422, 5425] [5054, 5423, 5425] [4887.]
5425 [4887, 4888, 4898, 4899, 4911, 5355, 5422, 5425] [5054, 5423, 5425] [4887. 4888.]
5425 [4887, 4888, 4898, 4899, 4911, 5355, 5422, 5425] [5054, 5423, 5425] [4887. 4888. 4898.]
5425 [4887, 4888, 4898, 4899, 4911, 5355, 5422, 

5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] []
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476. 4478.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476. 4478. 4479.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476. 4478. 4479. 4420.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476. 4478. 4479. 4420. 4476.]
5454 [4472, 4476, 4478, 4479, 5454] [4420, 4476, 4477, 4481, 4482, 4484, 4485, 4490, 5301, 5454] [4472. 4476. 4478. 4479. 4420. 4476. 4477.]
5454 [4472, 4476, 4478, 4479, 5454] [44

In [9]:
road_line_nodes.iloc[269]

Start Latitude     -11.396
Start Longitude     34.033
End Latitude       -11.393
End Longitude       34.033
Connection(s)      [262.0]
Name: 269, dtype: object

In [10]:
road_line_nodes

,Start Latitude,Start Longitude,End Latitude,End Longitude,Connection(s)
0,-11.391,33.985,-11.369,33.984,[]
1,-11.397,33.957,-11.394,34.048,[60.0]
2,-11.391,33.960,-11.400,33.963,[773.0]
3,-11.393,33.989,-11.410,33.993,"[98.0, 991.0]"
4,-11.388,34.046,-11.388,34.045,[]
...,...,...,...,...,...
5458,-11.449,33.971,-11.449,33.970,"[4735.0, 4737.0, 4738.0, 4744.0, 4746.0, 4747...."
5459,-11.450,33.970,-11.450,33.970,"[4351.0, 4549.0, 4740.0, 4742.0, 4752.0, 4753...."
5460,-11.437,33.964,-11.436,33.964,"[4355.0, 4357.0]"
5461,-11.446,33.967,-11.445,33.965,"[4931.0, 4317.0]"
